In [ ]:
pip install statsmodels

In [ ]:
pip install matplotlib

In [ ]:
pip install pandas

In [ ]:
pip install scikit-learn

In [ ]:
pip install scikit-learn

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
# Load the dataset
data = pd.read_excel("C:\\Users\\A B Siddik\\Desktop\\ARIMA\\data.xlsx")
data.head()

In [ ]:
# Filter the data to include only the relevant dates
filtered_data = data[(data['Date'] >= '2021-09-12') & (data['Date'] <= '2021-09-18')]

# Extract the 'New COVID-19 Cases' column
new_cases_vector_actual_data = filtered_data['New COVID-19 Cases'].values

print(new_cases_vector_actual_dat

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt

# === Step 1: Load and preprocess data ===
# Convert 'Date' column to datetime
data['Date'] = pd.to_datetime(data['Date'], dayfirst=True)

# Filter dataset between Jan 4, 2021, and Sep 18, 2021
data_filtered = data[(data['Date'] >= '2021-01-04') & (data['Date'] <= '2021-09-18')].reset_index(drop=True)

# Keep original Date column separately
dates = data_filtered['Date']

# Normalize all columns except Date
scaler = MinMaxScaler()
features = data_filtered.drop(columns=['Date'])
normalized_features = pd.DataFrame(scaler.fit_transform(features), columns=features.columns)
normalized_features['Date'] = dates

# === Step 2: Split into train/val/test ===
train_set = normalized_features[(normalized_features['Date'] >= '2021-01-04') & (normalized_features['Date'] <= '2021-07-02')]
val_set = normalized_features[(normalized_features['Date'] >= '2021-07-03') & (normalized_features['Date'] <= '2021-08-10')]
test_set = normalized_features[(normalized_features['Date'] >= '2021-08-11') & (normalized_features['Date'] <= '2021-09-18')]

# Reset index for each
train_set = train_set.reset_index(drop=True)
val_set = val_set.reset_index(drop=True)
test_set = test_set.reset_index(drop=True)

# === Step 3: Time series model on actual cases ===
# We'll use unnormalized actual cases for ARIMA forecasting
covid_cases_ts = data_filtered.set_index('Date')['New COVID-19 Cases']  # Replace 'Cases' with your actual column name

# Train ARIMA model on training + validation set
arima_train_series = covid_cases_ts['2021-01-04':'2021-08-10']
arima_model = ARIMA(arima_train_series, order=(6,1,6))  # You can optimize (p,d,q) or use auto_arima
arima_model = arima_model.fit()

# Print the model summary
print(arima_model.summary())


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import pandas as pd

# Step 4: Forecast and evaluate
test_data = covid_cases_ts['2021-08-11':'2021-09-18']
forecast = arima_model.forecast(steps=len(test_data))
forecast = pd.Series(forecast, index=test_data.index)

# Metrics
mse = mean_squared_error(test_data, forecast)
mae = mean_absolute_error(test_data, forecast)
mape = np.mean(np.abs((test_data - forecast) / test_data)) * 100
rmse = np.sqrt(mse)
rrmse = rmse / (np.max(test_data) - np.min(test_data))
nrmse = rmse / np.mean(test_data)  # Normalized RMSE by mean
r2 = r2_score(test_data, forecast)

# Print results
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Relative RMSE (RRMSE): {rrmse:.4f}")
print(f"Normalized RMSE (NRMSE): {nrmse:.4f}")
print(f"R-squared (R²): {r2:.4f}")


In [ ]:
from statsmodels.tsa.arima.model import ARIMA
import pandas as pd

# Initialize history with actual training values (no datetime index)
history = list(arima_train_series.values)

start_date = '2021-09-12'
end_date = '2021-09-18'

rolling_forecast = []

for date in pd.date_range(start=start_date, end=end_date):
    model = ARIMA(history, order=(6, 1, 6), enforce_stationarity=False, enforce_invertibility=False)
    model_fit = model.fit()

    output = model_fit.forecast(steps=1)  # output is np.ndarray
    next_value = output[0]                # fixed line
    rolling_forecast.append(next_value)
    
    history.append(next_value)            # add predicted value to history

# Convert to Series with date index
forecast_index = pd.date_range(start=start_date, end=end_date)
rolling_forecast_arima_with_no_exo = pd.Series(rolling_forecast, index=forecast_index)

# Optional: print or plot
print(rolling_forecast_arima_with_no_exo)


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Define the dates
dates = pd.date_range(start='2021-09-12', end='2021-09-18')

# Define the vectors
new_cases_vector_actual_data = [19550, 19198, 16073, 15669, 19495, 18815,17577]
forecast_vector_arima_no_exo = [17695.577209, 18454.495591, 19303.898229, 19389.129789, 18751.381991, 17731.829261, 17116.208056]
# Plot the data
plt.figure(figsize=(14, 8))
plt.plot(dates, new_cases_vector_actual_data, label='Actual active cases', marker='o')
plt.plot(dates, forecast_vector_arima_no_exo, label='ARIMA prediction ', marker='o')


# Add title and labels
plt.title('New active cases: Actual vs. Forecasted')
plt.xlabel('Date')
plt.ylabel('Number of new cases')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()

# Show plot
plt.show()